### Import Libraries

In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import gensim
from rake_nltk import Rake 

### Read in CSV

In [2]:
df = pd.read_csv("data/features_appstore_games.csv")
df.sample(3)

,url,id,name,subtitle,icon_url,average_user_rating,user_rating_count,price,in-app_purchases,description,...,combo_text,is_it_free,in-app_purch_sum,has_in-app_purch,successful_app,tokens,stems,lems,keywords,categories
15134,https://apps.apple.com/us/app/kaka-three-kingd...,1436114372,KAKA Three Kingdoms,,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,0.0,0.0,0.0,0.00,Description\nAs a card strategy game based on ...,...,kaka three kingdoms description\nas a card s...,0,0.0,0,0,"['kaka', 'three', 'kingdoms', 'description', '...","['kaka', 'three', 'kingdom', 'descript', 'na',...","['kaka', 'three', 'kingdom', 'description', 'n...","['win', 'card', 'strategy', 'game', 'based', '...","['Games', ' Strategy', ' Card']"
12335,https://apps.apple.com/us/app/rolling-ball-in-...,1263374980,Rolling Ball In The Cup,,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,0.0,0.0,0.0,0.00,"""Rolling Ball In The Cup is a pen color puzzle...",...,"rolling ball in the cup ""rolling ball in the...",0,0.0,0,0,"['rolling', 'ball', 'in', 'the', 'cup', 'rolli...","['roll', 'ball', 'in', 'the', 'cup', 'roll', '...","['rolling', 'ball', 'in', 'the', 'cup', 'rolli...","['meet', 'many', 'obstacles', 'make', 'solve',...","['Games', ' Entertainment', ' Puzzle', ' Strat..."
11649,https://apps.apple.com/us/app/infinit-bike-run...,1232577839,Infinit Bike Runner,,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,0.0,0.0,0.0,0.00,its an amazing game. its survival game of 2017...,...,infinit bike runner its an amazing game. its...,0,0.0,0,0,"['infinit', 'bike', 'runner', 'its', 'an', 'am...","['infinit', 'bike', 'runner', 'it', 'an', 'ama...","['infinit', 'bike', 'runner', 'it', 'an', 'ama...","['let', 'see', 'long', 'twitter', 'tab', 'face...","['Games', ' Racing', ' Strategy']"


### Selecting apps with ratings

In [3]:
df = df.loc[df["user_rating_count"] > 0]

### User input title

In [4]:
title = "Candy Crush Saga"

### User input description

In [5]:
description = """Start playing Candy Crush Saga today – a legendary puzzle game loved by millions of players around the world.


With over a trillion levels played, this sweet match 3 puzzle game is one of the most popular mobile games of all time!


Switch and match Candies in this tasty puzzle adventure to progress to the next level for that sweet winning feeling! Solve puzzles with quick thinking and smart moves, and be rewarded with delicious rainbow-colored cascades and tasty candy combos!


Plan your moves by matching 3 or more candies in a row, using boosters wisely in order to overcome those extra sticky puzzles! Blast the chocolate and collect sweet candy across thousands of levels, guaranteed to have you craving more!


Candy Crush Saga features:


THE GAME THAT KEEPS YOU CRAVING MORE

Thousands of the best levels and puzzles in the Candy Kingdom and with more added every 2 weeks your sugar fix is never far away! 


MANY WAYS TO WIN REWARDS

Check back daily and spin the Daily Booster Wheel to receive free tasty rewards, and take part in time limited challenges to earn boosters to help you level up! 


 VARIETY OF SUGAR-COATED CHALLENGES

Sweet ways to play: Game modes including Target Score, Clear the Jelly, Collect the Ingredients and Order Mode


PLAY ALONE OR WITH FRIENDS

Get to the top of the leaderboard events and compare scores with friends and competitors!


Levels range from easy to hard for all adults to enjoy – accessible on-the-go, offline and online.

It's easy to sync the game between devices and unlock full game features when connected to the Internet or Wifi.

Follow us to get news and updates; facebook.com/CandyCrushSaga, Twitter @CandyCrushSaga, Youtube https://www.youtube.com/user/CandyCrushOfficial

Visit https://community.king.com/en/candy-crush-saga to access the Community and competitions!

Candy Crush Saga is completely free to play but some optional in-game items will require payment.

You can turn off the payment feature by disabling in-app purchases in your device’s settings.

By downloading this game you are agreeing to our terms of service; http://about.king.com/consumer-terms/terms


Have fun playing Candy Crush Saga the sweetest match 3 puzzle game around! 

If you enjoy playing Candy Crush Saga, you may also enjoy its sister puzzle games; Candy Crush Soda Saga, Candy Crush Jelly Saga and Candy Crush Friends Saga!"""

### Pre-Function

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
def tokens(combo):
    return tokenizer.tokenize(combo)

stemmer = PorterStemmer()
def stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

lemmatizer = WordNetLemmatizer()
def lemmatizing(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def keyword_func(combo_text):
    r = Rake()
    r.extract_keywords_from_text(combo_text)
    key_words_dict = r.get_word_degrees()
    return list(key_words_dict.keys())

### Gensim

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format('../../Gensim/lexvec.enwiki+newscrawl.300d.W.pos.vectors')

### The function

In [8]:
def evaluator(title, description):
    new_app_combo = title + " " + description
    new_app_combo = new_app_combo.lower()
#     new_app_tokens = tokenizer.tokenize(new_app_combo)
#     new_app_stems = [stemmer.stem(token) for token in new_app_tokens]
#     new_app_lems = [lemmatizer.lemmatize(token) for token in new_app_tokens]
#     print(new_app_combo)

    new_app_keyw = keyword_func(new_app_combo)
    
    new_app_keyw_f = []
    for w in new_app_keyw:
        new_app_words = list(filter(lambda x: x in model.vocab, w))
        new_app_words = "".join(new_app_words)
        new_app_keyw_f.append(new_app_words)
    
    for empty_str in range(new_app_keyw_f.count("")):
        new_app_keyw_f.remove("")
        
    for w in new_app_keyw_f:
        if w in model.vocab:
            pass
        else:
            new_app_keyw_f.remove(w)
    
#     print(new_app_keyw_f)
    
    word2vec_list = []
#     print(model.most_similar(new_app_keyw_f))
    for tup in model.most_similar(new_app_keyw_f):
            word, sim = tup
            word2vec_list.append(word)
    word2vec_list.extend(new_app_keyw_f)
    print(word2vec_list)
    app_title_list = []
    t_perc = 0.9
    threshold = round(t_perc * len(word2vec_list))
    print(f"The threshold is set to {t_perc} similarity, taking {threshold} words of {len(word2vec_list)}")
    
    t_perc_min = 0.25
    while len(app_title_list) < 5 and threshold > round(t_perc_min * len(word2vec_list)):
        for app in df.loc[df['keywords'].map(lambda x:sum(1 for w in word2vec_list if w in x))>=threshold,'id']:
            app_title_list.append(app)
        for item in app_title_list:
            if app_title_list.count(item) > 1:
                app_title_list.reverse()
                app_title_list.remove(item)
                app_title_list.reverse()
        print(f"There are {len(app_title_list)} at a {threshold} threshold")
        threshold -= round(0.05 * len(word2vec_list))
        print(len(app_title_list))
    return app_title_list[:10] if len(app_title_list) > 0 else print('No Results!')

In [9]:
evaluator(title, description)

['and', 'just', 'all', 'choose', 'only', 'hopefully', 'customize', 'them', 'figuring', 'look', 'row', 'trillion', 'levels', 'played', 'visit', 'https', 'community', 'easy', 'match', 'candies', 'youtube', 'www', 'candy', 'crush', 'saga', 'com', 'matching', 'delicious', 'rainbow', 'adults', 'friends', 'moves', 'collect', 'sweet', 'puzzle', 'game', 'winning', 'feeling', 'get', 'news', 'guaranteed', 'one', 'soda', 'fun', 'playing', 'take', 'part', 'blast', 'variety', 'http', 'turn', 'play', 'may', 'also', 'enjoy', 'competitions', 'devices', 'receive', 'free', 'tasty', 'rewards', 'hard', 'players', 'around', 'follow', 'us', 'app', 'purchases', 'jelly', 'help', 'accessible', 'many', 'ways', 'next', 'level', 'overcome', 'ingredients', 'craving', 'popular', 'mobile', 'games', 'clear', 'keeps', 'facebook', 'sweetest', 'range', 'require', 'payment', 'wifi', 'never', 'far', 'away', 'modes', 'including', 'target', 'score', 'world', 'rewarded', 'chocolate', 'agreeing', 'settings', 'using', 'booster

[1154114309,
 965629604,
 965869424,
 1212982720,
 307132353,
 321756558,
 322110989,
 329218549,
 329466722,
 335029050]

## Building the application

In [10]:
the_list = [1154114309,
 965629604,
 965869424,
 1212982720,
 307132353,
 321756558,
 322110989,
 329218549,
 329466722,
 335029050]


### Extract titles

In [11]:
str(df.loc[df["id"] == the_list[0], "name"]).split("    ")[1].split("\n")[0]

'Kuros Classic'

In [12]:
names = []
for i in range(len(the_list)):
    name = str(df.loc[df["id"] == the_list[i], "name"]).split("    ")[1].split("\n")[0]
    names.append(name)
names

['Kuros Classic',
 'Gem Quest - Jewel Games Puzzle',
 'Super Gem Quest 2 Blast Mania',
 'Cute Safari: Match 3 Puzzle',
 '"Sally\'s Spa"',
 'Pixel Starships\\u2122',
 'PUSHY',
 'Chess - Play & Learn',
 'DodgeDot',
 'Catan Classic']

### Extract average user rating

In [13]:
float(df.loc[df["id"] == the_list[0], "average_user_rating"])

3.5

In [14]:
rating_list = []
rating_avg = 0
for i in range(len(the_list)):
    num = float(df.loc[df["id"] == the_list[i], "average_user_rating"])
    rating_list.append(num)
    rating_avg += num
rating_avg = rating_avg/len(the_list)
rating_list, rating_avg

([3.5, 5.0, 5.0, 5.0, 4.5, 4.5, 4.0, 4.5, 3.0, 4.0], 4.3)

### Create new DataFrame to use for extraction in the application

In [15]:
new_df = pd.DataFrame()
new_df

""


In [16]:
for sim_app in range(len(the_list)):
    if sim_app == 0:
        new_df = df.loc[df["id"] == the_list[sim_app]]
    else:
        app = df.loc[df["id"] == the_list[sim_app]]
        new_df = pd.concat([new_df, app])

new_df

### Create a DataFrame for top apps

In [17]:
a = df.sort_values("user_rating_count", ascending = False)[:10]
a

,url,id,name,subtitle,icon_url,average_user_rating,user_rating_count,price,in-app_purchases,description,...,combo_text,is_it_free,in-app_purch_sum,has_in-app_purch,successful_app,tokens,stems,lems,keywords,categories
1377,https://apps.apple.com/us/app/clash-of-clans/i...,529479190,Clash of Clans,Build your empire,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,4.5,3032734.0,0.0,"4.99, 9.99, 19.99, 0.99, 4.99, 0.99, 49.99, 99...","""Join millions of players worldwide as you bui...",...,"clash of clans build your empire ""join million...",0,199.90,1,1,"['clash', 'of', 'clans', 'build', 'your', 'emp...","['clash', 'of', 'clan', 'build', 'your', 'empi...","['clash', 'of', 'clan', 'build', 'your', 'empi...","['service', 'raise', 'also', 'u2019s', 'guide'...","['Games', ' Action', ' Entertainment', ' Strat..."
7185,https://apps.apple.com/us/app/clash-royale/id1...,1053012308,Clash Royale,Epic Real-Time Card Battles!,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,4.5,1277095.0,0.0,"4.99, 0.99, 9.99, 19.99, 9.99, 4.99, 0.99, 19....","""Enter the Arena! From the creators of Clash o...",...,"clash royale epic real-time card battles! ""ent...",0,77.90,1,1,"['clash', 'royale', 'epic', 'real', 'time', 'c...","['clash', 'royal', 'epic', 'real', 'time', 'ca...","['clash', 'royale', 'epic', 'real', 'time', 'c...","['ultimate', 'battle', 'deck', 'service', 'u25...","['Games', ' Strategy', ' Entertainment', ' Act..."
13410,https://apps.apple.com/us/app/pubg-mobile/id13...,1330123889,PUBG MOBILE,Team up! New 4v4 Mode is here,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,4.5,711409.0,0.0,"0.99, 9.99, 0.99, 9.99, 24.99, 4.99, 2.99, 99....",The official mobile version of PlayerUnknown\'...,...,pubg mobile team up! new 4v4 mode is here the ...,0,218.88,1,1,"['pubg', 'mobile', 'team', 'up', 'new', '4v4',...","['pubg', 'mobil', 'team', 'up', 'new', '4v4', ...","['pubg', 'mobile', 'team', 'up', 'new', '4v4',...","['r', '100', 'confirmed', 'pubgm', 'primeplus'...","['Games', ' Action', ' Strategy']"
1920,https://apps.apple.com/us/app/plants-vs-zombie...,597986893,Plants vs. Zombies\u2122 2,Save your brains!,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,4.5,469562.0,0.0,"4.99, 2.99, 9.99, 9.99, 4.99, 3.99, 3.99, 1.99...","""As beautifully presented as it is absorbing t...",...,"plants vs. zombies\u2122 2 save your brains! ""...",0,49.90,1,1,"['plants', 'vs', 'zombies', 'u2122', '2', 'sav...","['plant', 'vs', 'zombi', 'u2122', '2', 'save',...","['plant', 'v', 'zombie', 'u2122', '2', 'save',...","['service', 'u2019', 'notice', 'posted', 'stra...","['Games', ' Strategy', ' Entertainment', ' Adv..."
8137,https://apps.apple.com/us/app/pok%C3%A9mon-go/...,1094591345,Pok\xe9mon GO,Discover Pok\xe9mon worldwide,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,3.5,439776.0,0.0,"0.99, 4.99, 9.99, 19.99, 39.99, 99.99",NEW! Team GO Rocket has invaded the world of P...,...,pok\xe9mon go discover pok\xe9mon worldwide ne...,0,175.94,1,0,"['pok', 'xe9mon', 'go', 'discover', 'pok', 'xe...","['pok', 'xe9mon', 'go', 'discov', 'pok', 'xe9m...","['pok', 'xe9mon', 'go', 'discover', 'pok', 'xe...","['named', 'team', 'go', 'rocket', 'u201cbest',...","['Games', ' Strategy', ' Role Playing', ' Heal..."
2409,https://apps.apple.com/us/app/boom-beach/id672...,672150402,Boom Beach,Build. Plan. Boom!,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,4.5,400787.0,0.0,"4.99, 9.99, 19.99, 99.99, 49.99, 2.99, 0.99, 9.99","""Welcome to Boom Beach: come with a plan or le...",...,"boom beach build. plan. boom! ""welcome to boom...",0,198.92,1,1,"['boom', 'beach', 'build', 'plan', 'boom', 'we...","['boom', 'beach', 'build', 'plan', 'boom', 'we...","['boom', 'beach', 'build', 'plan', 'boom', 'we...","['base', 'allowed', 'extra', 'builders', 'cost...","['Games', ' Strategy', ' Action']"
12469,https://apps.apple.com/us/app/cash-inc-fame-fo...,1270598321,"Cash, Inc. Fame & Fortune Game",Fun Strategy Business Clicker!,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,5.0,374772.0,0.0

### Age Rating

In [18]:
int(str(df.loc[df["id"] == the_list[0], "age_rating"]).split("    ")[1].split("\n")[0].replace("+", ""))

4

### Genre Recommend

In [19]:
str(df.loc[df["id"] == the_list[0], "primary_genre"]).split("    ")[1].split("\n")[0]

'Games'

### Category Recommend

In [20]:
df.loc[df["id"] == the_list[0], "genres"]

9703    Games, Strategy, Puzzle
Name: genres, dtype: object

In [27]:
def app_categ(df, id_list):
    categ_list = []
    for i in range(len(the_list)):
        for elem in list(df.loc[df["id"] == the_list[i], "categories"])[0]:
            if elem in categ_list:
                pass
            else:
                categ_list.append(elem)
    return categ_list

In [28]:
app_categ(df, the_list)

['Games',
 ' Strategy',
 ' Puzzle',
 ' Entertainment',
 ' Action',
 ' Role Playing',
 ' Education',
 ' Board',
 ' Casual']

### Keep Lists after reading in the csv

In [23]:
import ast
def eval_literal(list_words):
    try:
        return ast.literal_eval(list_words)  
    except ValueError:
        return ast.literal_eval(str(list_words))  
# Items from list of strings to literal - ''
df['categories'] = df['categories'].map(eval_literal)

### Price range recommendation

In [24]:
e, r = float(df.loc[df["id"] == the_list[4], "price"]), 2

In [25]:
e

2.99

In [26]:
r

2